In [1]:
import pandas as pd
from itertools import product

# Carregar Dados

In [31]:
df_venda = pd.read_csv('../data/tb_vendas.csv')
df_venda.rename(columns={"cod_material": "cod_produto"}, inplace=True)

,key_loja,cod_produto
0,1,1428
119,1,1640
158,1,1650
207,1,22230
208,1,25458
...,...,...
202771,8,83772
202789,8,83845
203032,8,84203
203606,8,84204


In [32]:
df_estoque = pd.read_csv('../data/tb_estoques.csv')
df_estoque['dt_estoque'] = pd.to_datetime(df_estoque['dt_estoque'])
df_estoque.head()

,key_loja,cod_produto,dt_estoque,qt_estoque
0,1,1428,2024-03-18,108
1,1,1428,2024-12-13,24
2,1,1428,2024-07-30,33
3,1,1428,2024-07-22,34
4,1,1428,2024-07-29,34


In [69]:
df_estoque.shape

(260966, 4)

# Base de Ruptura

In [53]:
df_estoque['dt_estoque'] = pd.to_datetime(df_estoque['dt_estoque'])

primeira_data_estoque = (
    df_estoque
    .groupby(['key_loja', 'cod_produto'])['dt_estoque']
    .min()
    .reset_index()
    .rename(columns={'dt_estoque': 'primeira_data'})
)

primeira_data_estoque

,key_loja,cod_produto,primeira_data
0,1,1428,2024-01-01
1,1,1640,2024-01-01
2,1,1650,2024-01-01
3,1,22230,2024-01-01
4,1,24499,2024-01-01
...,...,...,...
826,8,84068,2024-01-01
827,8,84203,2024-01-01
828,8,84204,2024-01-01
829,8,84206,2024-01-01


In [55]:
primeira_data_estoque['primeira_data'].value_counts()

primeira_data
2024-01-01    693
2024-06-04    101
2024-02-25     14
2024-07-01      8
2024-08-17      8
2024-09-12      7
Name: count, dtype: int64

## Criando Grade de Loja + Produto + Dt Estoque

- Pego os valores unicos de produtos e lojas
- Gero uma lista de de datas desde a menor data que tenho até a maior para garantir que vamos ter todas as datas na nossa grade

In [61]:
produtos = df_estoque['cod_produto'].unique()
lojas = df_estoque['key_loja'].unique()
datas = pd.date_range(start=df_estoque['dt_estoque'].min(), end=df_estoque['dt_estoque'].max())

grade = pd.DataFrame(product(lojas, produtos, datas), columns=['key_loja', 'cod_produto', 'dt_estoque'])
grade = grade.merge(primeira_data_estoque, on=['key_loja', 'cod_produto'], how='left')
grade = grade[grade['dt_estoque'] >= grade['primeira_data']]
grade_estoque = grade.merge(df_estoque, on=['key_loja', 'cod_produto', 'dt_estoque'], how='left')
grade_estoque

,key_loja,cod_produto,dt_estoque,primeira_data,qt_estoque
0,1,1428,2024-01-01,2024-01-01,108.0
1,1,1428,2024-01-02,2024-01-01,108.0
2,1,1428,2024-01-03,2024-01-01,108.0
3,1,1428,2024-01-04,2024-01-01,108.0
4,1,1428,2024-01-05,2024-01-01,108.0
...,...,...,...,...,...
282643,8,85124,2024-12-27,2024-08-17,0.0
282644,8,85124,2024-12-28,2024-08-17,0.0
282645,8,85124,2024-12-29,2024-08-17,0.0
282646,8,85124,2024-12-30,2024-08-17,0.0


In [62]:
grade_estoque[grade_estoque['qt_estoque'].isnull()]

,key_loja,cod_produto,dt_estoque,primeira_data,qt_estoque
1255,1,22230,2024-06-06,2024-01-01,NaN
1256,1,22230,2024-06-07,2024-01-01,NaN
1257,1,22230,2024-06-08,2024-01-01,NaN
1258,1,22230,2024-06-09,2024-01-01,NaN
1259,1,22230,2024-06-10,2024-01-01,NaN
...,...,...,...,...,...
281289,8,84068,2024-08-30,2024-01-01,NaN
281290,8,84068,2024-08-31,2024-01-01,NaN
281291,8,84068,2024-09-01,2024-01-01,NaN
281292,8,84068,2024-09-02,2024-01-01,NaN


In [68]:
grade_estoque['is_ruptura'] = grade_estoque['qt_estoque'].isna() | (grade_estoque['qt_estoque'] == 0)
grade_estoque['is_ruptura'].value_counts()

is_ruptura
False    169462
True     113186
Name: count, dtype: int64

In [67]:
grade_estoque.to_csv('../data/dados_ruptura.csv', index=False)